# 🎯 Lezione 23 — DBSCAN: Clustering Basato su Densità

## Obiettivi di Apprendimento

| # | Obiettivo | Livello |
|---|-----------|---------|
| 1 | Capire la differenza tra clustering basato su densità e distanza | 🟢 Base |
| 2 | Comprendere i parametri `eps` e `min_samples` | 🟢 Base |
| 3 | Identificare core points, border points e noise | 🟡 Intermedio |
| 4 | Usare il k-distance graph per scegliere eps | 🟡 Intermedio |
| 5 | Applicare DBSCAN a dati con forme complesse | 🟡 Intermedio |
| 6 | Confrontare DBSCAN con K-Means e Gerarchico | 🔴 Avanzato |

---

## 📚 Indice

1. **Teoria** — Densità, parametri, tipi di punti
2. **Schema Mentale** — Workflow DBSCAN
3. **Demo Pratiche** — 5 demo progressive
4. **Esercizi** — 3 esercizi con soluzioni
5. **Conclusione** — Cosa portarsi a casa
6. **Bignami** — Reference card

---

## 🔧 Setup

```python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.neighbors import NearestNeighbors
```

---

## 📖 1. Teoria

### 1.1 Perché DBSCAN?

**Limiti di K-Means e Gerarchico:**
- Richiedono K a priori (o comunque una decisione)
- Assumono cluster **sferici/convessi**
- Ogni punto DEVE appartenere a un cluster

**DBSCAN risolve tutto questo:**
- **Non richiede K** — trova automaticamente il numero di cluster
- **Forme arbitrarie** — può trovare cluster di qualsiasi forma
- **Identifica outliers** — i punti "rumore" sono etichettati come -1

```
DBSCAN = Density-Based Spatial Clustering of Applications with Noise
```

### 📊 Confronto Visivo

```
K-Means:                    DBSCAN:
    ●●●     ○○○                ●●●●●●
   ●●●●●   ○○○○○              ●●    ●●
    ●●●     ○○○              ●●      ●●
                              ●●●●●●●
  (cluster sferici)         (cluster a forma di banana)
                              
                              ✗ ← outlier (noise=-1)
```

---

### 1.2 I Due Parametri Fondamentali

DBSCAN ha solo **2 parametri** (invece di K):

| Parametro | Nome | Significato |
|-----------|------|-------------|
| **eps** (ε) | Epsilon | Raggio del vicinato di un punto |
| **min_samples** | Minimo campioni | Numero minimo di punti nel vicinato per essere "denso" |

```
                    eps = raggio
                         ↓
                    ┌─────────┐
                    │    ●    │  Se ci sono ≥ min_samples punti
                    │  ● ◉ ●  │  in questo cerchio, ◉ è un CORE POINT
                    │    ●    │
                    └─────────┘
```

### 📐 Intuizione

- **eps piccolo** → molti cluster piccoli + molto noise
- **eps grande** → pochi cluster grandi, rischio di unire tutto
- **min_samples basso (es. 2)** → sensibile al rumore
- **min_samples alto (es. 10)** → solo zone molto dense sono cluster

---

### 1.3 I Tre Tipi di Punti

DBSCAN classifica ogni punto in una di tre categorie:

| Tipo | Definizione | Simbolo |
|------|-------------|---------|
| **Core Point** | Ha ≥ min_samples punti nel suo vicinato (raggio eps) | ◉ |
| **Border Point** | Non è core, ma è nel vicinato di un core point | ● |
| **Noise Point** | Non è core, non è nel vicinato di nessun core | ✗ |

```
Esempio con eps=1, min_samples=3:

     ✗ ← noise (isolato)
    
   ●     ← border (vicino a core ma < 3 vicini)
   ◉●●   ← core (ha 3+ vicini)
   ●◉●●  ← core (ha 4+ vicini)
    ●●
    
   ✗ ← noise
```

### 🔑 Regola Chiave

> **Un cluster è formato da tutti i core points connessi tra loro 
> (entro distanza eps) + i loro border points.**

---

### 1.4 L'Algoritmo Passo-Passo

```
ALGORITMO DBSCAN:

1. Per ogni punto p non ancora visitato:
   a. Marca p come visitato
   b. Trova tutti i punti nel vicinato di p (distanza ≤ eps)
   c. Se |vicinato| < min_samples → marca p come NOISE (temporaneamente)
   d. Altrimenti:
      - Crea nuovo cluster C
      - Aggiungi p a C (è un core point)
      - Per ogni punto q nel vicinato di p:
        * Se q non visitato → visita q e aggiungi i suoi vicini
        * Se q non appartiene a nessun cluster → aggiungi q a C

2. I punti noise rimasti alla fine sono gli outliers (label = -1)
```

### ⏱️ Complessità

| Caso | Complessità |
|------|-------------|
| **Con indicizzazione** (KD-tree, Ball-tree) | O(n log n) |
| **Senza indicizzazione** | O(n²) |

> **Nota:** sklearn usa automaticamente indicizzazione efficiente!

---

### 1.5 Come Scegliere eps: Il k-Distance Graph

Il trucco per scegliere **eps** è usare il **k-distance graph**:

1. Per ogni punto, calcola la distanza al k-esimo vicino più vicino (k = min_samples)
2. Ordina queste distanze in ordine crescente
3. Plotta: il "gomito" indica un buon valore di eps

```
Distanza                    
    |                    ●
    |                  ●
    |               ●●
    |           ●●●   ← gomito (knee)
    |      ●●●●       
    |  ●●●●
    |●●●
    +------------------------→ Punti (ordinati)
    
    Il valore di eps al gomito è una buona scelta!
```

### 📐 Formula per min_samples

Una regola empirica comune:
$$min\_samples \geq D + 1$$

dove D è la dimensionalità dei dati. Per 2D: min_samples ≥ 3

---

### 1.6 DBSCAN vs K-Means vs Gerarchico

| Caratteristica | K-Means | Gerarchico | DBSCAN |
|----------------|---------|------------|--------|
| **Richiede K** | ✅ Sì | ✅ Sì (taglio) | ❌ No |
| **Forma cluster** | Sferica | Dipende da linkage | Qualsiasi |
| **Gestisce outliers** | ❌ No | ❌ No | ✅ Sì (label=-1) |
| **Scalabilità** | ⭐⭐⭐ | ⭐ | ⭐⭐ |
| **Cluster di densità variabile** | ❌ | ❌ | ⚠️ Parziale |
| **Riproducibilità** | ⚠️ (init random) | ✅ | ✅ |

### 🎯 Quando Usare DBSCAN

✅ **Usa DBSCAN quando:**
- Non sai quanti cluster ci sono
- I cluster hanno forme non sferiche
- Ci sono outliers nei dati
- I cluster hanno densità simile

❌ **Evita DBSCAN quando:**
- I cluster hanno densità molto diverse
- I dati sono ad alta dimensionalità (curse of dimensionality)
- Hai bisogno di assegnare TUTTI i punti a un cluster

---

## 🧠 2. Schema Mentale

### Workflow DBSCAN

```
┌─────────────────────────────────────────────────────────────────────┐
│                        DBSCAN WORKFLOW                              │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  1. PREPARAZIONE                                                    │
│     └── StandardScaler (FONDAMENTALE per distanze!)                 │
│                                                                     │
│  2. STIMA min_samples                                               │
│     └── Regola: min_samples ≥ dimensioni + 1                        │
│     └── Per 2D: min_samples = 4 o 5 è un buon default               │
│                                                                     │
│  3. STIMA eps (k-distance graph)                                    │
│     └── NearestNeighbors(n_neighbors=min_samples)                   │
│     └── Calcola distanze al k-esimo vicino                          │
│     └── Ordina e plotta → trova il GOMITO                           │
│                                                                     │
│  4. FIT DBSCAN                                                      │
│     └── DBSCAN(eps=..., min_samples=...)                            │
│     └── labels_ contiene: 0, 1, 2... e -1 per noise                 │
│                                                                     │
│  5. VALUTAZIONE                                                     │
│     └── Numero cluster trovati: len(set(labels)) - (1 if -1 in      │
│         labels else 0)                                              │
│     └── Percentuale noise: (labels == -1).sum() / len(labels)       │
│     └── Silhouette (escludi noise!): silhouette_score(X[mask],      │
│         labels[mask])                                               │
│                                                                     │
│  6. TUNING (se necessario)                                          │
│     └── Troppi cluster? → aumenta eps                               │
│     └── Troppo noise? → aumenta eps o diminuisci min_samples        │
│     └── Cluster uniti? → diminuisci eps                             │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### ✅ Checklist Pre-DBSCAN

```
□ Dati scalati con StandardScaler?
□ min_samples ≥ dimensioni + 1?
□ Generato k-distance graph per stimare eps?
□ Pronti a gestire label -1 (noise)?
```

---

## 🔬 3. Demo Pratiche

### Demo 1 — Primo DBSCAN: Moon Dataset

Usiamo il classico dataset "moons" dove K-Means fallisce miseramente.

In [ ]:
# ============================================
# DEMO 1 — Primo DBSCAN: Moon Dataset
# ============================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN, KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.neighbors import NearestNeighbors

print("="*70)
print("DEMO 1 — DBSCAN vs K-Means su Moons Dataset")
print("="*70)

# Genera dataset "moons" (due mezzelune intrecciate)
np.random.seed(42)
X_moons, y_true = make_moons(n_samples=300, noise=0.05)

# Scaling (importante!)
scaler = StandardScaler()
X_moons_scaled = scaler.fit_transform(X_moons)

print(f"📊 Dataset: {len(X_moons)} punti, 2 cluster a forma di mezzaluna")

# Confronto: K-Means vs DBSCAN
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 1. Dati originali
axes[0].scatter(X_moons[:, 0], X_moons[:, 1], c=y_true, cmap='viridis', s=30, alpha=0.7)
axes[0].set_title('Ground Truth', fontsize=12)
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')

# 2. K-Means
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
labels_kmeans = kmeans.fit_predict(X_moons_scaled)
ari_kmeans = adjusted_rand_score(y_true, labels_kmeans)

axes[1].scatter(X_moons[:, 0], X_moons[:, 1], c=labels_kmeans, cmap='viridis', s=30, alpha=0.7)
axes[1].scatter(kmeans.cluster_centers_[:, 0] * scaler.scale_[0] + scaler.mean_[0],
                kmeans.cluster_centers_[:, 1] * scaler.scale_[1] + scaler.mean_[1],
                c='red', marker='X', s=200, edgecolors='black')
axes[1].set_title(f'K-Means (K=2)\nARI={ari_kmeans:.3f} ❌', fontsize=12)
axes[1].set_xlabel('x')

# 3. DBSCAN
dbscan = DBSCAN(eps=0.3, min_samples=5)
labels_dbscan = dbscan.fit_predict(X_moons_scaled)
ari_dbscan = adjusted_rand_score(y_true, labels_dbscan)

# Colori: noise in grigio
colors = labels_dbscan.copy().astype(float)
colors[labels_dbscan == -1] = -0.5  # Noise in grigio

scatter = axes[2].scatter(X_moons[:, 0], X_moons[:, 1], c=colors, cmap='viridis', s=30, alpha=0.7)
# Evidenzia noise
noise_mask = labels_dbscan == -1
if noise_mask.sum() > 0:
    axes[2].scatter(X_moons[noise_mask, 0], X_moons[noise_mask, 1], 
                    c='red', marker='x', s=50, label=f'Noise ({noise_mask.sum()})')
    axes[2].legend()
axes[2].set_title(f'DBSCAN (eps=0.3, min_samples=5)\nARI={ari_dbscan:.3f} ✅', fontsize=12)
axes[2].set_xlabel('x')

plt.tight_layout()
plt.show()

# Statistiche DBSCAN
n_clusters = len(set(labels_dbscan)) - (1 if -1 in labels_dbscan else 0)
n_noise = (labels_dbscan == -1).sum()

print(f"""
📊 RISULTATI:

K-Means:
   - ARI: {ari_kmeans:.3f} (basso = clustering sbagliato!)
   - Problema: assume cluster sferici

DBSCAN:
   - Cluster trovati: {n_clusters}
   - Punti noise: {n_noise} ({100*n_noise/len(X_moons):.1f}%)
   - ARI: {ari_dbscan:.3f} (alto = clustering corretto!)

🎯 DBSCAN trova le forme complesse dove K-Means fallisce!
""")

---

### Demo 2 — Il k-Distance Graph per Scegliere eps

Come trovare il valore ottimale di eps? Usiamo il metodo del "gomito" sul k-distance graph.

In [ ]:
# ============================================
# DEMO 2 — k-Distance Graph per Scegliere eps
# ============================================

print("="*70)
print("DEMO 2 — k-Distance Graph per Scegliere eps")
print("="*70)

# Genera un dataset con outliers
np.random.seed(42)
X_blobs, y_blobs = make_blobs(n_samples=250, centers=3, cluster_std=0.8, random_state=42)
# Aggiungi outliers
outliers = np.random.uniform(low=-10, high=10, size=(15, 2))
X_with_outliers = np.vstack([X_blobs, outliers])

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_with_outliers)

print(f"📊 Dataset: {len(X_blobs)} punti + {len(outliers)} outliers")

# ============================================
# PASSO 1: Calcola k-distanze
# ============================================
min_samples = 5  # Regola: D + 1 = 2 + 1 = 3, usiamo 5 per robustezza

# Trova il k-esimo vicino più vicino per ogni punto
nn = NearestNeighbors(n_neighbors=min_samples)
nn.fit(X_scaled)
distances, indices = nn.kneighbors(X_scaled)

# Prendi la distanza al k-esimo vicino (ultima colonna)
k_distances = distances[:, -1]

# Ordina in ordine crescente
k_distances_sorted = np.sort(k_distances)

# ============================================
# PASSO 2: Plot k-distance graph
# ============================================
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# k-distance graph
axes[0].plot(range(len(k_distances_sorted)), k_distances_sorted, 'b-', linewidth=2)
axes[0].set_xlabel('Punti (ordinati per k-distanza)', fontsize=11)
axes[0].set_ylabel(f'Distanza al {min_samples}° vicino', fontsize=11)
axes[0].set_title('k-Distance Graph\n(Cerca il GOMITO)', fontsize=12)
axes[0].grid(True, alpha=0.3)

# Trova il "gomito" (approssimazione: derivata seconda massima)
# Metodo semplice: cerca dove la pendenza cambia di più
second_derivative = np.diff(np.diff(k_distances_sorted))
knee_idx = np.argmax(second_derivative) + 2  # +2 per offset
eps_optimal = k_distances_sorted[knee_idx]

axes[0].axhline(y=eps_optimal, color='red', linestyle='--', linewidth=2, 
                label=f'eps ottimale ≈ {eps_optimal:.2f}')
axes[0].scatter([knee_idx], [eps_optimal], color='red', s=100, zorder=5, marker='o')
axes[0].legend()

# ============================================
# PASSO 3: Applica DBSCAN con eps stimato
# ============================================
dbscan = DBSCAN(eps=eps_optimal, min_samples=min_samples)
labels = dbscan.fit_predict(X_scaled)

n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise = (labels == -1).sum()

# Plot clustering
colors = plt.cm.viridis(np.linspace(0, 1, n_clusters + 1))
for i in range(n_clusters):
    mask = labels == i
    axes[1].scatter(X_with_outliers[mask, 0], X_with_outliers[mask, 1], 
                    c=[colors[i]], s=50, alpha=0.7, label=f'Cluster {i}')

# Noise in rosso
noise_mask = labels == -1
axes[1].scatter(X_with_outliers[noise_mask, 0], X_with_outliers[noise_mask, 1],
                c='red', marker='x', s=80, label=f'Noise ({n_noise})')

axes[1].set_xlabel('x', fontsize=11)
axes[1].set_ylabel('y', fontsize=11)
axes[1].set_title(f'DBSCAN (eps={eps_optimal:.2f}, min_samples={min_samples})\n'
                  f'{n_clusters} cluster, {n_noise} noise points', fontsize=12)
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"""
📊 RISULTATI:

1. k-Distance Graph:
   - min_samples = {min_samples}
   - eps ottimale (dal gomito) ≈ {eps_optimal:.2f}

2. DBSCAN:
   - Cluster trovati: {n_clusters}
   - Punti noise: {n_noise} ({100*n_noise/len(X_with_outliers):.1f}%)
   
🎯 Gli outliers aggiunti sono stati identificati come NOISE!
""")

---

### Demo 3 — Core Points, Border Points, Noise

Visualizziamo i tre tipi di punti che DBSCAN identifica.

In [ ]:
# ============================================
# DEMO 3 — Core Points, Border Points, Noise
# ============================================

print("="*70)
print("DEMO 3 — Core Points, Border Points, Noise")
print("="*70)

# Dataset semplice per visualizzazione chiara
np.random.seed(123)
X_small = np.array([
    # Cluster 1 (denso)
    [1, 1], [1.2, 0.8], [0.8, 1.2], [1.1, 1.1], [0.9, 0.9],
    [1.3, 1.0], [1.0, 1.3],
    # Cluster 2 (denso)
    [4, 4], [4.2, 3.8], [3.8, 4.2], [4.1, 4.1], [3.9, 3.9],
    [4.3, 4.0], [4.0, 4.3],
    # Border points (al limite)
    [2.0, 2.0],  # Border tra i cluster
    [0.3, 0.3],  # Border di cluster 1
    # Noise (isolati)
    [6, 1],
    [0, 5],
    [7, 7]
])

eps = 0.8
min_samples = 3

print(f"📊 Dataset: {len(X_small)} punti")
print(f"   Parametri: eps={eps}, min_samples={min_samples}")

# Fit DBSCAN
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
labels = dbscan.fit_predict(X_small)

# Identifica core samples (sklearn li salva in core_sample_indices_)
core_mask = np.zeros(len(X_small), dtype=bool)
core_mask[dbscan.core_sample_indices_] = True

# Border = non-core ma con label != -1
border_mask = ~core_mask & (labels != -1)

# Noise = label == -1
noise_mask = labels == -1

print(f"\n📌 Classificazione punti:")
print(f"   Core points: {core_mask.sum()}")
print(f"   Border points: {border_mask.sum()}")
print(f"   Noise points: {noise_mask.sum()}")

# Visualizzazione
fig, ax = plt.subplots(figsize=(10, 8))

# Core points (grandi e pieni)
ax.scatter(X_small[core_mask, 0], X_small[core_mask, 1], 
           c='blue', s=200, marker='o', edgecolors='black', linewidths=2,
           label=f'Core Points ({core_mask.sum()})', alpha=0.8)

# Border points (medi)
ax.scatter(X_small[border_mask, 0], X_small[border_mask, 1],
           c='green', s=150, marker='s', edgecolors='black', linewidths=2,
           label=f'Border Points ({border_mask.sum()})', alpha=0.8)

# Noise points (X rosse)
ax.scatter(X_small[noise_mask, 0], X_small[noise_mask, 1],
           c='red', s=150, marker='X', edgecolors='black', linewidths=2,
           label=f'Noise Points ({noise_mask.sum()})', alpha=0.8)

# Disegna cerchi eps intorno ai core points
for i, (x, y) in enumerate(X_small[core_mask]):
    circle = plt.Circle((x, y), eps, fill=False, color='blue', 
                         linestyle='--', alpha=0.3)
    ax.add_patch(circle)

# Etichette punti
for i, (x, y) in enumerate(X_small):
    tipo = "C" if core_mask[i] else ("B" if border_mask[i] else "N")
    ax.annotate(f'{i}({tipo})', (x + 0.1, y + 0.1), fontsize=8)

ax.set_xlabel('x', fontsize=11)
ax.set_ylabel('y', fontsize=11)
ax.set_title(f'DBSCAN: Core, Border, Noise\n(eps={eps}, min_samples={min_samples})', fontsize=13)
ax.legend(loc='upper right', fontsize=10)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.set_xlim(-1, 9)
ax.set_ylim(-1, 9)

plt.tight_layout()
plt.show()

print(f"""
📖 LEGENDA:
   C = Core Point (≥ {min_samples} vicini entro eps={eps})
   B = Border Point (nel vicinato di un core, ma < {min_samples} vicini)
   N = Noise Point (isolato)
   
   I cerchi tratteggiati mostrano il raggio eps intorno ai core points.
""")

---

### Demo 4 — Effetto dei Parametri eps e min_samples

Esploriamo come cambiano i risultati variando i due parametri.

In [ ]:
# ============================================
# DEMO 4 — Effetto dei Parametri eps e min_samples
# ============================================

print("="*70)
print("DEMO 4 — Effetto dei Parametri eps e min_samples")
print("="*70)

# Dataset: cerchi concentrici (sfida per K-Means!)
np.random.seed(42)
X_circles, y_circles = make_circles(n_samples=300, noise=0.05, factor=0.5)
scaler = StandardScaler()
X_circles_scaled = scaler.fit_transform(X_circles)

# Griglia di parametri
eps_values = [0.1, 0.3, 0.5]
min_samples_values = [3, 5, 10]

fig, axes = plt.subplots(3, 3, figsize=(12, 12))

for i, eps in enumerate(eps_values):
    for j, min_s in enumerate(min_samples_values):
        dbscan = DBSCAN(eps=eps, min_samples=min_s)
        labels = dbscan.fit_predict(X_circles_scaled)
        
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise = (labels == -1).sum()
        pct_noise = 100 * n_noise / len(X_circles)
        
        # Plot
        ax = axes[i, j]
        
        # Colori per cluster
        unique_labels = set(labels)
        colors = plt.cm.viridis(np.linspace(0, 1, len(unique_labels)))
        
        for k, col in zip(unique_labels, colors):
            if k == -1:
                # Noise
                col = 'red'
                marker = 'x'
                size = 30
            else:
                marker = 'o'
                size = 20
            
            mask = labels == k
            ax.scatter(X_circles[mask, 0], X_circles[mask, 1], 
                      c=[col], marker=marker, s=size, alpha=0.7)
        
        ax.set_title(f'eps={eps}, min_s={min_s}\n{n_clusters} cluster, {pct_noise:.0f}% noise',
                     fontsize=10)
        ax.set_xticks([])
        ax.set_yticks([])
        
        # Evidenzia casi interessanti
        if n_clusters == 2 and pct_noise < 5:
            ax.patch.set_edgecolor('green')
            ax.patch.set_linewidth(4)

# Etichette assi
for i, eps in enumerate(eps_values):
    axes[i, 0].set_ylabel(f'eps={eps}', fontsize=12, fontweight='bold')
for j, min_s in enumerate(min_samples_values):
    axes[0, j].set_title(f'min_samples={min_s}\n' + axes[0, j].get_title(), fontsize=10)

plt.suptitle('Effetto di eps e min_samples su Cerchi Concentrici\n(bordo verde = configurazione ottimale)',
             fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("""
📊 OSSERVAZIONI:

eps PICCOLO (0.1):
   → Molti cluster piccoli o tutto noise
   → Troppo restrittivo

eps MEDIO (0.3):
   → Trova i 2 cerchi correttamente ✅
   → Poco noise

eps GRANDE (0.5):
   → Rischia di unire i cerchi
   → Meno noise ma clustering sbagliato

min_samples BASSO (3):
   → Sensibile al rumore
   → Più punti considerati "densi"

min_samples ALTO (10):
   → Solo zone molto dense sono cluster
   → Più punti diventano noise
""")

---

### Demo 5 — DBSCAN vs K-Means vs Gerarchico: Confronto Finale

Confrontiamo i tre metodi di clustering su un dataset con forme complesse.

In [ ]:
# ============================================
# DEMO 5 — DBSCAN vs K-Means vs Gerarchico
# ============================================
from sklearn.cluster import AgglomerativeClustering

print("="*70)
print("DEMO 5 — Confronto Finale: DBSCAN vs K-Means vs Gerarchico")
print("="*70)

# Dataset combinato: moons + blob + outliers
np.random.seed(42)

# Due mezzelune
X_moons, y_moons = make_moons(n_samples=200, noise=0.05)
X_moons[:, 0] += 3  # Sposta a destra

# Un blob
X_blob = np.random.normal(loc=[-2, 0], scale=0.5, size=(100, 2))

# Outliers
outliers = np.array([[-4, 3], [6, 3], [2, -2], [-3, -2], [5, -1]])

# Combina
X_complex = np.vstack([X_moons, X_blob, outliers])
y_true = np.concatenate([y_moons, np.full(100, 2), np.full(5, -1)])

# Scaling
scaler = StandardScaler()
X_complex_scaled = scaler.fit_transform(X_complex)

print(f"📊 Dataset complesso:")
print(f"   - 2 mezzelune (200 punti)")
print(f"   - 1 blob (100 punti)")
print(f"   - 5 outliers")
print(f"   - Totale: {len(X_complex)} punti")

# ============================================
# Confronto metodi
# ============================================
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. Ground Truth
ax = axes[0, 0]
colors_true = y_true.copy().astype(float)
colors_true[y_true == -1] = 3  # outliers
scatter = ax.scatter(X_complex[:, 0], X_complex[:, 1], c=colors_true, 
                      cmap='viridis', s=30, alpha=0.7)
ax.scatter(outliers[:, 0], outliers[:, 1], c='red', marker='X', s=100, 
           edgecolors='black', label='Outliers')
ax.set_title('Ground Truth\n(3 cluster + 5 outliers)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

# 2. K-Means
ax = axes[0, 1]
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
labels_km = kmeans.fit_predict(X_complex_scaled)
ari_km = adjusted_rand_score(y_true[y_true != -1], labels_km[y_true != -1])
ax.scatter(X_complex[:, 0], X_complex[:, 1], c=labels_km, cmap='viridis', s=30, alpha=0.7)
ax.set_title(f'K-Means (K=3)\nARI={ari_km:.3f}', fontsize=12)
ax.grid(True, alpha=0.3)

# 3. Gerarchico
ax = axes[1, 0]
agg = AgglomerativeClustering(n_clusters=3, linkage='ward')
labels_agg = agg.fit_predict(X_complex_scaled)
ari_agg = adjusted_rand_score(y_true[y_true != -1], labels_agg[y_true != -1])
ax.scatter(X_complex[:, 0], X_complex[:, 1], c=labels_agg, cmap='viridis', s=30, alpha=0.7)
ax.set_title(f'Gerarchico (Ward, K=3)\nARI={ari_agg:.3f}', fontsize=12)
ax.grid(True, alpha=0.3)

# 4. DBSCAN
ax = axes[1, 1]
dbscan = DBSCAN(eps=0.3, min_samples=5)
labels_db = dbscan.fit_predict(X_complex_scaled)
n_clusters_db = len(set(labels_db)) - (1 if -1 in labels_db else 0)
n_noise_db = (labels_db == -1).sum()

# Calcola ARI escludendo noise
mask_valid = (y_true != -1) & (labels_db != -1)
if mask_valid.sum() > 0:
    ari_db = adjusted_rand_score(y_true[mask_valid], labels_db[mask_valid])
else:
    ari_db = 0

# Colori
colors_db = labels_db.astype(float)
colors_db[labels_db == -1] = -1

scatter = ax.scatter(X_complex[labels_db != -1, 0], X_complex[labels_db != -1, 1], 
                      c=labels_db[labels_db != -1], cmap='viridis', s=30, alpha=0.7)
ax.scatter(X_complex[labels_db == -1, 0], X_complex[labels_db == -1, 1],
           c='red', marker='x', s=50, label=f'Noise ({n_noise_db})')
ax.set_title(f'DBSCAN (eps=0.3, min_samples=5)\n{n_clusters_db} cluster, ARI={ari_db:.3f}', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

for ax in axes.flat:
    ax.set_xlabel('x')
    ax.set_ylabel('y')

plt.suptitle('Confronto Metodi di Clustering su Dataset Complesso', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

# Tabella riassuntiva
print("\n" + "="*60)
print("TABELLA RIASSUNTIVA")
print("="*60)
print(f"\n{'Metodo':<20} {'ARI':>10} {'Gestisce Outliers':>20}")
print("-"*60)
print(f"{'K-Means':<20} {ari_km:>10.3f} {'❌ No':>20}")
print(f"{'Gerarchico':<20} {ari_agg:>10.3f} {'❌ No':>20}")
print(f"{'DBSCAN':<20} {ari_db:>10.3f} {'✅ Sì ('+str(n_noise_db)+' trovati)':>20}")
print("-"*60)

print(f"""
🎯 CONCLUSIONE:

DBSCAN è l'unico che:
   1. Trova cluster di forme non sferiche (le mezzelune)
   2. Identifica gli outliers automaticamente
   3. Non richiede K a priori
""")

---

## 📝 4. Esercizi

### 📝 Esercizio 23.1 — Anomaly Detection con DBSCAN

**Consegna:** Usa DBSCAN per identificare transazioni anomale in un dataset di e-commerce.

**Dataset:**
```python
importo = [25, 30, 28, 35, 500, 22, 27, 1000, 33, 29, 31, 26, 750, 28, 24]
durata_sessione = [5, 6, 4, 7, 2, 5, 6, 1, 8, 5, 6, 4, 3, 5, 6]  # minuti
```

**Richieste:**
1. Scala i dati con StandardScaler
2. Usa il k-distance graph per scegliere eps (min_samples=3)
3. Applica DBSCAN e identifica le transazioni anomale (noise)
4. Interpreta: perché queste transazioni sono considerate anomale?

In [ ]:
# ============================================
# ESERCIZIO 23.1 — SOLUZIONE
# ============================================

print("="*70)
print("ESERCIZIO 23.1 — Anomaly Detection con DBSCAN")
print("="*70)

# ============================================
# PASSO 1: Preparazione dati
# ============================================
importo = [25, 30, 28, 35, 500, 22, 27, 1000, 33, 29, 31, 26, 750, 28, 24]
durata_sessione = [5, 6, 4, 7, 2, 5, 6, 1, 8, 5, 6, 4, 3, 5, 6]

df_trans = pd.DataFrame({
    'transazione': [f'T{i}' for i in range(1, 16)],
    'importo': importo,
    'durata_min': durata_sessione
})

print("\n📊 Dataset Transazioni:")
print(df_trans)

X_trans = df_trans[['importo', 'durata_min']].values
scaler = StandardScaler()
X_trans_scaled = scaler.fit_transform(X_trans)

# ============================================
# PASSO 2: k-distance graph
# ============================================
min_samples = 3

nn = NearestNeighbors(n_neighbors=min_samples)
nn.fit(X_trans_scaled)
distances, _ = nn.kneighbors(X_trans_scaled)
k_distances = np.sort(distances[:, -1])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# k-distance graph
axes[0].plot(range(len(k_distances)), k_distances, 'b-o', linewidth=2, markersize=6)
axes[0].set_xlabel('Punti (ordinati)', fontsize=11)
axes[0].set_ylabel(f'Distanza al {min_samples}° vicino', fontsize=11)
axes[0].set_title('k-Distance Graph', fontsize=12)
axes[0].grid(True, alpha=0.3)

# Trova gomito (approssimazione)
second_deriv = np.diff(np.diff(k_distances))
knee_idx = np.argmax(second_deriv) + 2
eps_optimal = k_distances[knee_idx]

axes[0].axhline(y=eps_optimal, color='red', linestyle='--', linewidth=2,
                label=f'eps ≈ {eps_optimal:.2f}')
axes[0].legend()

# ============================================
# PASSO 3: Applica DBSCAN
# ============================================
# Uso un eps leggermente più conservativo per catturare anomalie
eps_used = 0.8  # Dopo analisi del graph

dbscan = DBSCAN(eps=eps_used, min_samples=min_samples)
labels = dbscan.fit_predict(X_trans_scaled)

df_trans['cluster'] = labels
df_trans['anomalia'] = labels == -1

# Plot
colors = ['green' if l != -1 else 'red' for l in labels]
axes[1].scatter(df_trans['importo'], df_trans['durata_min'], 
                c=colors, s=150, edgecolors='black', alpha=0.8)

for i, row in df_trans.iterrows():
    marker = '⚠️' if row['anomalia'] else ''
    axes[1].annotate(f"{row['transazione']}{marker}", 
                     (row['importo']+20, row['durata_min']+0.1), fontsize=9)

axes[1].set_xlabel('Importo (€)', fontsize=11)
axes[1].set_ylabel('Durata Sessione (min)', fontsize=11)
axes[1].set_title(f'DBSCAN (eps={eps_used}, min_samples={min_samples})\n'
                  f'Rosso = Anomalie', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================
# PASSO 4: Interpretazione
# ============================================
print("\n" + "="*70)
print("PASSO 4: Interpretazione Anomalie")
print("="*70)

anomalie = df_trans[df_trans['anomalia']]
normali = df_trans[~df_trans['anomalia']]

print(f"\n📊 STATISTICHE:")
print(f"\n   Transazioni normali ({len(normali)}):")
print(f"   - Importo medio: €{normali['importo'].mean():.0f}")
print(f"   - Durata media: {normali['durata_min'].mean():.1f} min")

print(f"\n   🚨 ANOMALIE RILEVATE ({len(anomalie)}):")
for _, row in anomalie.iterrows():
    print(f"\n   {row['transazione']}:")
    print(f"   - Importo: €{row['importo']} (vs media €{normali['importo'].mean():.0f})")
    print(f"   - Durata: {row['durata_min']} min (vs media {normali['durata_min'].mean():.1f} min)")
    
    # Interpretazione
    if row['importo'] > 400:
        print("   → SOSPETTO: Importo anomalmente alto + sessione breve")
        print("   → Possibile frode o errore di sistema")

print("\n✅ DBSCAN ha identificato automaticamente le transazioni sospette!")

---

### 📝 Esercizio 23.2 — Clustering Geografico

**Consegna:** Usa DBSCAN per raggruppare punti di interesse in una città.

**Dataset:**
```python
lat = [45.46, 45.47, 45.46, 45.48, 45.70, 45.71, 45.69, 45.20, 45.46, 45.47]
lon = [9.18, 9.19, 9.17, 9.18, 9.30, 9.31, 9.29, 9.50, 9.20, 9.18]
nomi = ['Duomo', 'Scala', 'Castello', 'Brera', 'Monza1', 'Monza2', 'Monza3', 
        'Pavia', 'Navigli', 'Porta Romana']
```

**Richieste:**
1. Identifica le zone di interesse (cluster di POI vicini)
2. Trova eventuali POI isolati
3. Assegna nomi descrittivi alle zone trovate

In [ ]:
# ============================================
# ESERCIZIO 23.2 — SOLUZIONE
# ============================================

print("="*70)
print("ESERCIZIO 23.2 — Clustering Geografico POI")
print("="*70)

# ============================================
# PASSO 1: Preparazione dati
# ============================================
lat = [45.46, 45.47, 45.46, 45.48, 45.70, 45.71, 45.69, 45.20, 45.46, 45.47]
lon = [9.18, 9.19, 9.17, 9.18, 9.30, 9.31, 9.29, 9.50, 9.20, 9.18]
nomi = ['Duomo', 'Scala', 'Castello', 'Brera', 'Monza1', 'Monza2', 'Monza3', 
        'Pavia', 'Navigli', 'Porta Romana']

df_poi = pd.DataFrame({
    'nome': nomi,
    'lat': lat,
    'lon': lon
})

print("\n📍 Dataset POI:")
print(df_poi)

X_geo = df_poi[['lat', 'lon']].values

# Per dati geografici, scaling può alterare le proporzioni
# Usiamo i dati raw ma con eps appropriato
# 0.05 gradi ≈ 5km

# ============================================
# PASSO 2: k-distance graph
# ============================================
min_samples = 2  # Piccolo dataset

nn = NearestNeighbors(n_neighbors=min_samples)
nn.fit(X_geo)
distances, _ = nn.kneighbors(X_geo)
k_distances = np.sort(distances[:, -1])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(range(len(k_distances)), k_distances, 'b-o', linewidth=2, markersize=8)
axes[0].set_xlabel('Punti (ordinati)', fontsize=11)
axes[0].set_ylabel('Distanza al vicino', fontsize=11)
axes[0].set_title('k-Distance Graph (dati geografici)', fontsize=12)
axes[0].grid(True, alpha=0.3)

# eps = circa 0.05 (distanza tra POI nella stessa zona)
eps_geo = 0.05
axes[0].axhline(y=eps_geo, color='red', linestyle='--', label=f'eps={eps_geo}')
axes[0].legend()

# ============================================
# PASSO 3: Applica DBSCAN
# ============================================
dbscan = DBSCAN(eps=eps_geo, min_samples=min_samples)
labels = dbscan.fit_predict(X_geo)

df_poi['cluster'] = labels

n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise = (labels == -1).sum()

print(f"\n📊 DBSCAN: {n_clusters} zone trovate, {n_noise} POI isolati")

# Plot mappa
colors = plt.cm.Set1(np.linspace(0, 1, n_clusters + 1))

for i in range(n_clusters):
    mask = labels == i
    axes[1].scatter(df_poi.loc[mask, 'lon'], df_poi.loc[mask, 'lat'],
                    c=[colors[i]], s=200, edgecolors='black', 
                    label=f'Zona {i}', alpha=0.8)

# POI isolati
noise_mask = labels == -1
if noise_mask.sum() > 0:
    axes[1].scatter(df_poi.loc[noise_mask, 'lon'], df_poi.loc[noise_mask, 'lat'],
                    c='gray', s=200, marker='X', edgecolors='black',
                    label='Isolati', alpha=0.8)

# Etichette
for i, row in df_poi.iterrows():
    axes[1].annotate(row['nome'], (row['lon']+0.01, row['lat']+0.01), fontsize=9)

axes[1].set_xlabel('Longitudine', fontsize=11)
axes[1].set_ylabel('Latitudine', fontsize=11)
axes[1].set_title('Mappa POI con Clustering DBSCAN', fontsize=12)
axes[1].legend(loc='upper left')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================
# PASSO 4: Nomi descrittivi
# ============================================
print("\n" + "="*70)
print("PASSO 4: Zone Identificate")
print("="*70)

zone_nomi = {}
for cluster_id in sorted(df_poi['cluster'].unique()):
    if cluster_id == -1:
        continue
    
    cluster_data = df_poi[df_poi['cluster'] == cluster_id]
    poi_list = cluster_data['nome'].tolist()
    
    # Logica naming basata sui POI
    if 'Duomo' in poi_list or 'Scala' in poi_list:
        zona_nome = "🏛️ Centro Storico Milano"
    elif 'Monza' in poi_list[0]:
        zona_nome = "🏎️ Zona Monza"
    else:
        zona_nome = f"📍 Zona {cluster_id}"
    
    zone_nomi[cluster_id] = zona_nome
    print(f"\n   {zona_nome}:")
    print(f"   POI: {', '.join(poi_list)}")

# POI isolati
isolati = df_poi[df_poi['cluster'] == -1]
if len(isolati) > 0:
    print(f"\n   🚫 POI ISOLATI (non raggruppabili):")
    for _, row in isolati.iterrows():
        print(f"   - {row['nome']} ({row['lat']:.2f}, {row['lon']:.2f})")

print("\n✅ Clustering geografico completato!")

---

### 📝 Esercizio 23.3 — Tuning Automatico di eps

**Consegna:** Implementa una funzione che trova automaticamente il miglior eps per DBSCAN.

**Richieste:**
1. Crea una funzione `trova_eps_ottimale(X, min_samples)` che usa il k-distance graph
2. Testa la funzione sul dataset make_moons con noise
3. Confronta il risultato con eps scelti manualmente (0.1, 0.3, 0.5)
4. Valuta con Silhouette Score (escludi noise!)

In [ ]:
# ============================================
# ESERCIZIO 23.3 — SOLUZIONE
# ============================================

print("="*70)
print("ESERCIZIO 23.3 — Tuning Automatico di eps")
print("="*70)

# ============================================
# PASSO 1: Funzione trova_eps_ottimale
# ============================================

def trova_eps_ottimale(X, min_samples, plot=False):
    """
    Trova il valore ottimale di eps usando il metodo del gomito
    sul k-distance graph.
    
    Parameters:
    -----------
    X : array-like, shape (n_samples, n_features)
        Dati (già scalati!)
    min_samples : int
        Parametro min_samples per DBSCAN
    plot : bool
        Se True, mostra il k-distance graph
        
    Returns:
    --------
    eps_ottimale : float
        Valore di eps al gomito
    """
    # Calcola k-distanze
    nn = NearestNeighbors(n_neighbors=min_samples)
    nn.fit(X)
    distances, _ = nn.kneighbors(X)
    k_distances = np.sort(distances[:, -1])
    
    # Trova il gomito (massima curvatura)
    # Metodo: trova dove la derivata seconda è massima
    if len(k_distances) > 3:
        second_deriv = np.diff(np.diff(k_distances))
        knee_idx = np.argmax(second_deriv) + 2
    else:
        knee_idx = len(k_distances) // 2
    
    eps_ottimale = k_distances[knee_idx]
    
    if plot:
        plt.figure(figsize=(8, 5))
        plt.plot(range(len(k_distances)), k_distances, 'b-', linewidth=2)
        plt.axhline(y=eps_ottimale, color='red', linestyle='--', 
                    label=f'eps ottimale = {eps_ottimale:.3f}')
        plt.scatter([knee_idx], [eps_ottimale], color='red', s=100, zorder=5)
        plt.xlabel('Punti (ordinati)')
        plt.ylabel(f'Distanza al {min_samples}° vicino')
        plt.title('k-Distance Graph con Gomito Automatico')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
    
    return eps_ottimale

print("✅ Funzione trova_eps_ottimale() definita!")

# ============================================
# PASSO 2: Test su make_moons
# ============================================
print("\n" + "="*70)
print("PASSO 2: Test su make_moons")
print("="*70)

np.random.seed(42)
X_test, y_test = make_moons(n_samples=300, noise=0.08)
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

min_samples = 5
eps_auto = trova_eps_ottimale(X_test_scaled, min_samples, plot=True)
print(f"\n🎯 eps ottimale trovato automaticamente: {eps_auto:.3f}")

# ============================================
# PASSO 3: Confronto con eps manuali
# ============================================
print("\n" + "="*70)
print("PASSO 3: Confronto eps manuali vs automatico")
print("="*70)

eps_values = [0.1, 0.3, 0.5, eps_auto]
eps_labels = ['0.1 (piccolo)', '0.3 (medio)', '0.5 (grande)', f'{eps_auto:.3f} (auto)']

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

risultati = {}

for idx, (eps, label) in enumerate(zip(eps_values, eps_labels)):
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(X_test_scaled)
    
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = (labels == -1).sum()
    
    # Silhouette (escludi noise!)
    mask = labels != -1
    if mask.sum() > 1 and len(set(labels[mask])) > 1:
        sil = silhouette_score(X_test_scaled[mask], labels[mask])
    else:
        sil = -1
    
    # ARI
    ari = adjusted_rand_score(y_test, labels)
    
    risultati[label] = {
        'eps': eps,
        'n_clusters': n_clusters,
        'n_noise': n_noise,
        'silhouette': sil,
        'ari': ari
    }
    
    # Plot
    ax = axes[idx]
    colors = labels.astype(float)
    colors[labels == -1] = -0.5
    
    ax.scatter(X_test[labels != -1, 0], X_test[labels != -1, 1],
               c=labels[labels != -1], cmap='viridis', s=30, alpha=0.7)
    if n_noise > 0:
        ax.scatter(X_test[labels == -1, 0], X_test[labels == -1, 1],
                   c='red', marker='x', s=30, label=f'Noise ({n_noise})')
        ax.legend()
    
    ax.set_title(f'eps={label}\n{n_clusters} cluster, Sil={sil:.3f}, ARI={ari:.3f}',
                 fontsize=10)
    ax.grid(True, alpha=0.3)
    
    # Evidenzia il migliore
    if eps == eps_auto:
        ax.patch.set_edgecolor('green')
        ax.patch.set_linewidth(4)

plt.suptitle('Confronto eps: Manuale vs Automatico\n(bordo verde = automatico)', fontsize=13)
plt.tight_layout()
plt.show()

# ============================================
# PASSO 4: Tabella riassuntiva
# ============================================
print("\n" + "-"*70)
print(f"{'eps':<20} {'Cluster':>10} {'Noise':>10} {'Silhouette':>12} {'ARI':>10}")
print("-"*70)
for label, res in risultati.items():
    marker = "⭐" if 'auto' in label else "  "
    print(f"{marker}{label:<18} {res['n_clusters']:>10} {res['n_noise']:>10} "
          f"{res['silhouette']:>12.3f} {res['ari']:>10.3f}")
print("-"*70)

print("""
🎯 CONCLUSIONE:
   Il metodo automatico del gomito trova un eps ragionevole,
   ma potrebbe necessitare di fine-tuning manuale per casi specifici.
""")

---

## 🎯 5. Conclusione

### ✅ Cosa Portarsi a Casa

| Concetto | Cosa Ricordare |
|----------|----------------|
| **DBSCAN** | Clustering basato su densità, non richiede K |
| **eps** | Raggio del vicinato — usa k-distance graph per sceglierlo |
| **min_samples** | Minimo punti per essere "denso" — regola: ≥ D+1 |
| **Noise (label=-1)** | DBSCAN identifica automaticamente gli outliers |
| **Core/Border/Noise** | I tre tipi di punti che DBSCAN classifica |
| **Forme arbitrarie** | DBSCAN trova cluster di qualsiasi forma |

### ⚠️ Errori Comuni

| Errore | Perché è Sbagliato | Correzione |
|--------|-------------------|------------|
| Non scalare i dati | eps dipende dalla scala | Sempre StandardScaler |
| Ignorare il noise | Può contenere informazioni utili | Analizza i punti noise |
| eps troppo grande | Unisce cluster distinti | Usa k-distance graph |
| eps troppo piccolo | Tutto diventa noise | Aumenta gradualmente |
| Silhouette con noise | Risultato falsato | `silhouette_score(X[mask], labels[mask])` |

### 🔗 Ponte verso la Lezione 24

Nella prossima lezione esploreremo **PCA** (Principal Component Analysis):
- **Riduzione dimensionalità** — da N feature a K componenti
- **Varianza spiegata** — quanto informazione conserviamo
- **Visualizzazione** — proiettare dati in 2D/3D
- Combineremo PCA + Clustering nella Lezione 25

---

## 📚 6. Bignami — DBSCAN

### 📖 Definizioni Chiave

| Termine | Definizione |
|---------|-------------|
| **DBSCAN** | Density-Based Spatial Clustering of Applications with Noise |
| **eps (ε)** | Raggio del vicinato di un punto |
| **min_samples** | Numero minimo di punti nel vicinato per essere "denso" |
| **Core Point** | Punto con ≥ min_samples vicini entro eps |
| **Border Point** | Nel vicinato di un core point ma non è core |
| **Noise Point** | Non è core e non è nel vicinato di nessun core (label = -1) |
| **k-distance graph** | Grafico per scegliere eps (cercare il gomito) |

### 📐 Formule

| Concetto | Formula |
|----------|---------|
| Vicinato | $N_{eps}(p) = \{q \in D \mid dist(p, q) \leq eps\}$ |
| Core Point | $\|N_{eps}(p)\| \geq min\_samples$ |
| min_samples consigliato | $min\_samples \geq D + 1$ (D = dimensionalità) |

### ✅ Checklist Pre-DBSCAN

```
□ Dati scalati con StandardScaler?
□ min_samples ≥ dimensioni + 1?
□ Generato k-distance graph?
□ Trovato il gomito per eps?
□ Pronti a gestire label -1 (noise)?
□ Silhouette calcolato SENZA noise?
```

### 💻 Template di Codice

```python
# === DBSCAN COMPLETO ===
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score
import numpy as np

# 1. Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2. Stima eps con k-distance graph
min_samples = 5  # ≥ D + 1
nn = NearestNeighbors(n_neighbors=min_samples)
nn.fit(X_scaled)
distances, _ = nn.kneighbors(X_scaled)
k_distances = np.sort(distances[:, -1])

# Plot per trovare il gomito
import matplotlib.pyplot as plt
plt.plot(k_distances)
plt.xlabel('Punti')
plt.ylabel(f'Distanza al {min_samples}° vicino')
plt.show()

# 3. DBSCAN
eps = 0.5  # Dal gomito
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
labels = dbscan.fit_predict(X_scaled)

# 4. Statistiche
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise = (labels == -1).sum()
print(f"Cluster: {n_clusters}, Noise: {n_noise}")

# 5. Silhouette (ESCLUDI NOISE!)
mask = labels != -1
if mask.sum() > 1:
    sil = silhouette_score(X_scaled[mask], labels[mask])
    print(f"Silhouette: {sil:.3f}")

# 6. Core samples
core_mask = np.zeros(len(X), dtype=bool)
core_mask[dbscan.core_sample_indices_] = True
```

### 🎯 Quando Usare

| Usa DBSCAN quando... | Evita DBSCAN quando... |
|----------------------|------------------------|
| Non conosci K | I cluster hanno densità diverse |
| Ci sono outliers | Dati ad alta dimensionalità |
| Cluster non sferici | Tutti i punti DEVONO essere in un cluster |
| Vuoi anomaly detection | Hai bisogno di un modello predittivo |

---

✅ **Lezione 23 Completata!**